### LRCN Model

In [1]:
import pandas as pd
import os
import numpy as np
import cv2

In [2]:
labels_df = pd.read_csv("TrainLabels.csv")
labels_df.columns = labels_df.columns.str.strip()  # remove leading/trailing spaces
print(labels_df.columns.tolist())

['ClipID', 'Boredom', 'Engagement', 'Confusion', 'Frustration']


In [3]:
DATA_DIR = r"C:\Users\Randall Chiang\Documents\FYP\DAiSEE\Data\Train"

def get_frames_for_clip(clip_path, max_frames=30, img_size=(64, 64)):
    frames = []
    for fname in sorted(os.listdir(clip_path)):
        if fname.endswith(".jpg"):
            img_path = os.path.join(clip_path, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            frames.append(img)
    frames = np.array(frames)
    if len(frames) > max_frames:
        frames = frames[:max_frames]
    elif len(frames) < max_frames:
        pad = np.zeros((max_frames - len(frames), *img_size, 3), dtype=np.uint8)
        frames = np.vstack([frames, pad])
    return frames

X, y = [], []

for i, row in labels_df.iterrows():
    clip_id = row["ClipID"].replace(".avi", "")
    # Prefix folder is first 6 digits of clip_id
    prefix_folder = clip_id[:6]
    clip_path = os.path.join(DATA_DIR, prefix_folder, clip_id)

    if os.path.exists(clip_path) and os.path.isdir(clip_path):
        frames = get_frames_for_clip(clip_path)
        if frames.shape[0] > 0:
            X.append(frames)
            label_vec = [
                row["Boredom"],
                row["Engagement"],
                row["Confusion"],
                row["Frustration"]
            ]
            y.append(label_vec)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)  # (num_clips, max_frames, H, W, 3)
print("y shape:", y.shape)  # (num_clips, 4)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 4 dimension(s)

In [ ]:
np.save("X.npy", X)
np.save("Y.npy", y)